In [ ]:
import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional
import pandas as pd

In [ ]:
# Loading BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = TFBertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
df = pd.read_csv("//content//tweets_cleaned.csv", encoding = "latin1")
df.head()

,Unnamed: 0,Date,User,Tweet,Likes,Retweets,Cleaned_Tweet,Sentiment_Score,Sentiment
0,0,2023-03-29 15:42:36+00:00,AnandPatni8,@vinodkapri @RahulGandhi Respected Indian Citi...,0.0,0.0,rahulgandhi respected indian citizens namaskaa...,0.6705,Positive
1,1,2023-03-29 15:42:05+00:00,dhinamum,"*Respected Indian Citizens,* Namaskaar I Am Th...",0.0,0.0,respected indian citizens namaskaar original g...,0.6705,Positive
2,2,2023-03-29 15:34:29+00:00,PrincetonCGI,1/n-Meet Filmmaker Prakash Jha in New Jersey t...,0.0,0.0,1 n meet filmmaker prakash jha new jersey talk...,0.5267,Positive
3,3,2023-03-29 15:31:43+00:00,RishiJoeSanu,@MrinalWahal Why would politicians stop using ...,0.0,0.0,would politicians stop using religion politics...,-0.2960,Negative
4,4,2023-03-29 15:26:48+00:00,itweetsensee,@annamalai_k @narendramodi A state level presi...,0.0,0.0,narendramodi state level president knows polic...,-0.6369,Negative


In [ ]:
# Tokenizing and padding sequences
input_ids = tokenizer(df['Cleaned_Tweet'].tolist(), return_tensors='tf', padding=True, truncation=True)['input_ids']

# Convert labels to one-hot encoded vectors
labels = pd.get_dummies(df['Sentiment']).values


In [ ]:
model_lstm_bert = Sequential()
model_lstm_bert.add(Embedding(input_dim=bert_model.config.vocab_size, output_dim=768))
model_lstm_bert.add(Bidirectional(LSTM(128)))
model_lstm_bert.add(Dense(3, activation='softmax'))

model_lstm_bert.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train LSTM with BERT embeddings
model_lstm_bert.fit(input_ids, labels, epochs= 5, batch_size=32, validation_split=0.2)


Epoch 1/5
566/566 [==============================] - 408s 710ms/step - loss: 0.6779 - accuracy: 0.7043 - val_loss: 0.4920 - val_accuracy: 0.8112
Epoch 2/5
566/566 [==============================] - 385s 680ms/step - loss: 0.2780 - accuracy: 0.9016 - val_loss: 0.4850 - val_accuracy: 0.8145
Epoch 3/5
566/566 [==============================] - 382s 674ms/step - loss: 0.1325 - accuracy: 0.9548 - val_loss: 0.6075 - val_accuracy: 0.8095
Epoch 4/5
566/566 [==============================] - 382s 675ms/step - loss: 0.0715 - accuracy: 0.9772 - val_loss: 0.7362 - val_accuracy: 0.8033
Epoch 5/5
566/566 [==============================] - 395s 698ms/step - loss: 0.0371 - accuracy: 0.9880 - val_loss: 0.8962 - val_accuracy: 0.7953


In [ ]:
model_lstm_bert.save('my_model.h5')
# model_lstm_bert.save('my_model.keras')

/usr/local/lib/python3.10/dist-packages/keras/src/initializers/__init__.py:144: UserWarning: The `keras.initializers.serialize()` API should only be used for objects of type `keras.initializers.Initializer`. Found an instance of type <class 'tf_keras.src.initializers.initializers.GlorotUniform'>, which may lead to improper serialization.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/initializers/__init__.py:144: UserWarning: The `keras.initializers.serialize()` API should only be used for objects of type `keras.initializers.Initializer`. Found an instance of type <class 'tf_keras.src.initializers.initializers.Orthogonal'>, which may lead to improper serialization.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/initializers/__init__.py:144: UserWarning: The `keras.initializers.serialize()` API should only be used for objects of type `keras.initializers.Initializer`. Found an instance of type <class 'tf_keras.src.initializers.initializers.Zeros'>,

In [ ]:
from keras.models import load_model
model = load_model('my_model.h5')

In [ ]:
# Evaluating the model
accuracy = model_lstm_bert.evaluate(input_ids, labels)[1]
print("Accuracy: {:.2f}%".format(accuracy * 100))

707/707 [==============================] - 55s 78ms/step - loss: 0.1954 - accuracy: 0.9545
Accuracy: 95.45%


In [ ]:
# Loading the separate dataset
separate_df = pd.read_csv("//content//Twitter_Data.csv", encoding="latin1")
separate_df["category"] = separate_df["category"].replace({1: "Positive",-1: "Negative",0: "Neutral"})
separate_df.head()

,clean_text,category
0,when modi promised âminimum government maxim...,Negative
1,talk all the nonsense and continue all the dra...,Neutral
2,what did just say vote for modi welcome bjp t...,Positive
3,asking his supporters prefix chowkidar their n...,Positive
4,answer who among these the most powerful world...,Positive


In [ ]:
import pandas as pd
from transformers import BertTokenizer, TFBertForSequenceClassification
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, Bidirectional
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import re
import spacy

import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('vader_lexicon')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [ ]:
# Function to clean and preprocess text
def preprocess_text(text):
    if isinstance(text, str):
        # Removing special characters, URLs, and mentions
        text = re.sub(r"http\S+|@\S+|[^A-Za-z0-9]+", " ", text)

        # Converting to lowercase
        text = text.lower()

        # Removing stopwords
        stop_words = set(stopwords.words("english"))
        word_tokens = word_tokenize(text)
        filtered_text = [word for word in word_tokens if word not in stop_words]

        return " ".join(filtered_text)
    else:
        return ""

separate_df['Cleaned_Text'] = separate_df['clean_text'].apply(preprocess_text)
separate_df.head()

,clean_text,category,Cleaned_Text
0,when modi promised âminimum government maxim...,Negative,modi promised minimum government maximum gover...
1,talk all the nonsense and continue all the dra...,Neutral,talk nonsense continue drama vote modi
2,what did just say vote for modi welcome bjp t...,Positive,say vote modi welcome bjp told rahul main camp...
3,asking his supporters prefix chowkidar their n...,Positive,asking supporters prefix chowkidar names modi ...
4,answer who among these the most powerful world...,Positive,answer among powerful world leader today trump...


In [ ]:
# Loading spaCy
nlp = spacy.load("en_core_web_sm")

# Function to filter government-related tweets using Named Entity Recognition (NER)
def filter_government_tweets_ner(separate_df):
    government_entities = ['ORG', 'GPE', 'NORP']  # Organization, Geopolitical Entity, Nationality/Religious/Political Group
    government_related_tweets = []
    for tweet in separate_df['Cleaned_Text']:
        doc = nlp(tweet)
        entities = [ent.label_ for ent in doc.ents]
        if any(entity in government_entities for entity in entities):
            government_related_tweets.append(tweet)
    government_related_tweets_df = separate_df[separate_df['Cleaned_Text'].isin(government_related_tweets)]
    return government_related_tweets_df

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


In [ ]:
filter_tweets = filter_government_tweets_ner(separate_df)
filter_tweets

,clean_text,category,Cleaned_Text
6,surat women perform yagna seeks divine grace f...,Neutral,surat women perform yagna seeks divine grace n...
8,with upcoming election india saga going import...,Positive,upcoming election india saga going important p...
10,things like demonetisation gst goods and servi...,Positive,things like demonetisation gst goods services ...
13,one vote can make all the difference anil kapo...,Neutral,one vote make difference anil kapoor answers m...
14,one vote can make all the difference anil kapo...,Neutral,one vote make difference anil kapoor answers m...
...,...,...,...
162974,save your agenda peddling â had with terror ...,Neutral,save agenda peddling terror attacks every seco...
162975,why these 456 crores paid neerav modi not reco...,Negative,456 crores paid neerav modi recovered congress...
162976,dear rss terrorist payal gawar what about modi...,Negative,dear rss terrorist payal gawar modi killing 10...
162978,there big project came into india modi dream p...,Neutral,big project came india modi dream project happ...


In [ ]:
# filter_tweets.to_csv('filter_tweets.csv', index=False)
separate_df.info()
filter_tweets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 162980 entries, 0 to 162979
Data columns (total 3 columns):
 #   Column        Non-Null Count   Dtype 
---  ------        --------------   ----- 
 0   clean_text    162976 non-null  object
 1   category      162973 non-null  object
 2   Cleaned_Text  162980 non-null  object
dtypes: object(3)
memory usage: 3.7+ MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 64102 entries, 6 to 162979
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   clean_text    64102 non-null  object
 1   category      64100 non-null  object
 2   Cleaned_Text  64102 non-null  object
dtypes: object(3)
memory usage: 2.0+ MB


In [ ]:
# Tokenizing and padding sequences for the separate dataset
separate_input_ids = tokenizer(filter_tweets['Cleaned_Text'].tolist(), return_tensors='tf', padding=True, truncation=True)['input_ids']

# One-hot encode labels for the separate dataset
separate_labels = pd.get_dummies(filter_tweets['category']).values

In [ ]:
# Evaluate the model on the separate dataset
evaluation_results = model_lstm_bert.evaluate(separate_input_ids, separate_labels)

1603/1603 [==============================] - 52s 32ms/step - loss: 2.7850 - accuracy: 0.5355


In [ ]:
# Extracting evaluation metrics
loss = evaluation_results[0]
accuracy = evaluation_results[1]

print("Loss on separate dataset: {:.4f}".format(loss))
print("Accuracy on separate dataset: {:.2f}%".format(accuracy * 100))


Loss on separate dataset: 2.7850
Accuracy on separate dataset: 53.55%
